### 1. 问题
#### 【问题一】 上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

#pivot适用于在索引唯一且数据集较大的情况下，按照指定列(category)对特定数据列进行分组显示，不支持对数据进行统计计算
df_pivot = df.pivot(index='ID', columns='Gender', values='Height').head()

#pivot_table适用于同时按照索引和指定列(category)对特定数据列进行分组后统计计算
df.pivot_table(index='School', columns='Gender', values=['Height', 'Weight'], aggfunc=['max', 'min'], margins=True).head()

#crosstab适用于对不同表提供的索引和指定列(category)分组后对单个数据列进行统计计算以及对频数和百分比的快速计算
pd.crosstab(index=df['School'], columns=df['Gender'], values=df['Height'], aggfunc=['max', 'min'])

#melt适用于将pivot分组后的数据再融合成一列
df_pivot.reset_index().melt(id_vars=['ID'], value_vars=['F', 'M'], value_name='Height')

#stack适用于将列标签堆叠为索引或单独的一列
display(df.stack())

#unstack适用于将多层索引中的某一层展开为列标签
df.stack().unstack().head()


0   School          S_1
    Class           C_1
    ID             1101
    Gender            M
    Address    street_1
                 ...   
34  Address    street_6
    Height          193
    Weight           54
    Math           47.6
    Physics           B
Length: 315, dtype: object

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


### 2. 练习
#### 【练习一】 继续使用上一章的药物数据集：
#### (a) 现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：
![avatar](picture/drug_pic.png)

In [55]:
df = pd.read_csv('data/Drugs.csv')
df1 = df.pivot_table(values='DrugReports', index=['State', 'COUNTY', 'SubstanceName'], columns='YYYY', fill_value='-') \
.reset_index().rename_axis(columns={'YYYY':''})
display(df1.head())

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [77]:
#方法1
df2 = df1.replace({'-':np.nan}).set_index(['State', 'COUNTY', 'SubstanceName']).stack().reset_index(level=3)
df2.columns=['YYYY', 'DrugReports']
df2 = df2.reset_index().sort_values('YYYY', ignore_index=True)
cols = list(df2.columns)
cols.insert(0, cols.pop(cols.index('YYYY')))
df2 = df2.loc[:, cols].astype({'DrugReports':'int'})
display(df2.sort_values(['State','COUNTY','YYYY','SubstanceName'], ignore_index=True).equals(df.sort_values(['State','COUNTY','YYYY','SubstanceName'], ignore_index=True)))

#方法2
df3 = df1.reset_index().melt(id_vars=['State', 'COUNTY', 'SubstanceName'], value_vars=list(range(2010, 2018))
                       , value_name='DrugReports').replace({'-':np.nan}).dropna()
df3.columns = ['State', 'COUNTY', 'SubstanceName', 'YYYY', 'DrugReports']
cols = list(df3.columns)
cols.insert(0, cols.pop(cols.index('YYYY')))
df3 = df3.loc[:, cols].astype({'DrugReports':'int'})
display(df3.sort_values(['State','COUNTY','YYYY','SubstanceName'], ignore_index=True).equals(df.sort_values(['State','COUNTY','YYYY','SubstanceName'], ignore_index=True)))

True

True

#### 【练习二】 现有一份关于某地区地震情况的数据集，请解决如下问题：
#### (a) 现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：
![avatar](picture/earthquake_pic.png)

In [2]:
df = pd.read_csv('data/Earthquake.csv')
df1 = df.pivot_table(index=['日期','时间','维度','经度'], columns='方向').stack(level=0).replace({np.nan:'-'})
df1.index.names = ['日期', '时间', '维度', '经度', '地震参数']
df1

方向                                      east north north_east north_west  \
日期         时间          维度    经度    地震参数                                    
1912.08.09 12:29:00 AM 40.60 27.20 深度      -     -          -          -   
                                   烈度      -     -          -          -   
                                   距离      -     -          -          -   
1912.08.10 12:23:00 AM 40.60 27.10 深度      -     -          -          -   
                                   烈度      -     -          -          -   
...                                      ...   ...        ...        ...   
2017.07.30 12:21:33 AM 37.39 31.37 烈度      -     -          -          -   
                                   距离      -     -          -          -   
2017.07.31 12:11:31 AM 37.42 31.37 深度      -     -          -        5.1   
                                   烈度      -     -          -        3.5   
                                   距离      -     -          -        6.2   

方向                                      south south_east south_west west  
日期         时间          维度    经度    地震参数                                   
1912.08.09 12:29:00 AM 40.60 27.20 深度       -         16          -    -  
                                   烈度       -        6.7          -    -  
                                   距离       -        4.3          -    -  
1912.08.10 12:23:00 AM 40.60 27.10 深度       -          -         15    -  
                                   烈度       -          -          6    -  
...                                       ...        ...        ...  ...  
2017.07.30 12:21:33 AM 37.39 31.37 烈度       -          -        3.5    -  
                                   距离       -          -          6    -  
2017.07.31 12:11:31 AM 37.42 31.37 深度       -          -          -    -  
                                   烈度       -          -          -    -  
                                   距离       -          -          -    -  

[30186 rows x 8 columns]

#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [3]:
#df1.reset_index()
#display(df1.columns)
#df1.melt(id_vars=df1.columns[:5], value_vars=df1.columns[5:])
df2 = df1.stack().unstack(level=4).reset_index().replace({'-':np.nan}).dropna().rename_axis(columns={'地震参数':''})
cols = list(df2.columns)
cols.insert(5, cols.pop(cols.index('距离')))
df2 = df2.loc[:, cols]
display(df2.sort_values(list(df2.columns), ignore_index=True).equals(df.sort_values(list(df.columns), ignore_index=True)))

True